In [25]:
from transformers import pipeline
from transformers import AlbertTokenizer, AlbertForMaskedLM
import torch

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForMaskedLM.from_pretrained('albert-base-v2')

nlp = pipeline("fill-mask", model=model, tokenizer=tokenizer,topk=10)
#print(nlp(f"HuggingFace is creating a {nlp.tokenizer.mask_token} that the community uses to solve NLP tasks."))

In [31]:
results = nlp("HuggingFace is creating a [MASK] that the community uses to solve NLP tasks.")
results

[{'sequence': '[CLS] huggingface is creating a tool that the community uses to solve nlp tasks.[SEP]',
  'score': 0.2723134756088257,
  'token': 5607},
 {'sequence': '[CLS] huggingface is creating a platform that the community uses to solve nlp tasks.[SEP]',
  'score': 0.06568177044391632,
  'token': 2452},
 {'sequence': '[CLS] huggingface is creating a template that the community uses to solve nlp tasks.[SEP]',
  'score': 0.0644591748714447,
  'token': 22894},
 {'sequence': '[CLS] huggingface is creating a solution that the community uses to solve nlp tasks.[SEP]',
  'score': 0.03637254238128662,
  'token': 4295},
 {'sequence': '[CLS] huggingface is creating a resource that the community uses to solve nlp tasks.[SEP]',
  'score': 0.035429175943136215,
  'token': 6577},
 {'sequence': '[CLS] huggingface is creating a framework that the community uses to solve nlp tasks.[SEP]',
  'score': 0.035174690186977386,
  'token': 6596},
 {'sequence': '[CLS] huggingface is creating a strategy that

In [32]:
word = tokenizer.convert_ids_to_tokens(results[0]['token'])[1:]
word

'tool'

In [33]:
score = results[0]['score']
score

0.2723134756088257

In [20]:
model.eval()

AlbertForMaskedLM(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
  

In [42]:
from transformers import pipeline
import logging


def albert_masked_word_prediction(masked_examples, model, tokenizer, gpu_available, top_n, logger):
#     if gpu_available:
#         model.cuda()
#         logger.info("successfully moved model to gpu")

    model.eval()

    fill_mask_pipeline = pipeline("fill-mask", model=model, tokenizer=tokenizer, topk=top_n)
    
    avg_responses = {}
    for j, key in enumerate(masked_examples):
        binary_avg_score = 0
        ratio_avg_score = 0

        for example in masked_examples[key]:
            statement, right_answer, wrong_answer = example
            statement = statement.replace('<mask>','[MASK]')
            print(statement)
            responses = fill_mask_pipeline(statement)
            right_pos = top_n + 1
            wrong_pos = top_n + 1
            right_score = 0
            wrong_score = 0
            done = -1
            for i in range(len(responses)):
                possible_answer = tokenizer.convert_ids_to_tokens(responses[0]['token'])[1:]
                if possible_answer == right_answer:
                    right_score = responses[i]['score']
                    right_pos = i
                    done += 1
                if possible_answer == wrong_answer:
                    wrong_score = responses[i]['score']
                    wrong_pos = i
                    done += 1
                if done > 0:
                    break
            
            binary_score = 1 if right_pos < wrong_pos else 0
            
            if right_score + wrong_score > 0:
                ratio_score = (right_score - wrong_score) / (right_score + wrong_score)
            else:
                ratio_score = -1
            
            binary_avg_score += binary_score
            ratio_avg_score += ratio_score

        binary_avg_score /= float(len(masked_examples[key]))
        ratio_avg_score /= float(len(masked_examples[key]))

        avg_responses[key] = {"binary_score" : binary_avg_score, "ratio_score" : ratio_avg_score}
        
        if (j+1) % 240 == 0:
            logger.info("finished 10 more")

    return avg_responses

In [43]:
import random
random.seed(1012)
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

masked = {'0-original-original':[('Joioaw is smaller than Ilkl, so Joioaw is <mask> likely to fit into boxes than Ilkl.','more','less')]}

albert_masked_word_prediction(masked,model,tokenizer,False,10,logger)



Joioaw is smaller than Ilkl, so Joioaw is [MASK] likely to fit into boxes than Ilkl.


{'0-original-original': {'binary_score': 0.0, 'ratio_score': -1.0}}

In [44]:
statement

NameError: name 'statement' is not defined